In [1]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline 
import re

#nlp
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction import DictVectorizer


from scipy.sparse import hstack
from scipy import sparse

from sklearn.linear_model import Ridge
from sklearn import linear_model

from scipy import math, stats

# Loading data and cleaning data

In [2]:
#loading data
train = pd.read_table("/Users/Jpeteyy/PythonScripts/train.tsv", encoding = 'utf-8').head(50000)
test = pd.read_table("/Users/Jpeteyy/PythonScripts/test.tsv", encoding = 'utf-8').head(50000)
#test this later 48952

In [3]:
#splitting categories into multiple buckets
#train['cat1'], train['cat2'], train['cat3'] = train['category_name'].str.split('/', 2).str
for p,q in enumerate(['cat1','cat2','cat3']):
    train[q] = train['category_name'].apply(lambda x: x.split('/')[p] if pd.isnull(x) != True else '')
    
    
#splitting categories into multiple buckets
#train['cat1'], train['cat2'], train['cat3'] = train['category_name'].str.split('/', 2).str
for p,q in enumerate(['cat1','cat2','cat3']):
    test[q] = test['category_name'].apply(lambda x: x.split('/')[p] if pd.isnull(x) != True else '')

In [4]:
for k in ['item_description', 'name']:
    train[k] = train[k].apply(lambda x: str.lower(x) if type(x) == str else '')
    
    
for k in ['item_description', 'name']:
    test[k] = test[k].apply(lambda x: str.lower(x) if type(x) == str else '')

In [5]:
#replacing missing values function
def replace_nan(dataframe):
    #dataframe.category_name.fillna(value="missing", inplace=True)
    dataframe.brand_name.fillna(value="missing", inplace=True)
    #dataframe.item_description.fillna(value="missing", inplace=True)
    #dataframe.cat1.fillna(value="missing", inplace=True)
    #dataframe.cat2.fillna(value="missing", inplace=True)
    #dataframe.cat3.fillna(value="missing", inplace=True)
    #dataframe.cat4.fillna(value="missing", inplace=True)
    return (dataframe)

In [6]:
#passing training data into replacing missing func
train = replace_nan(train)

test = replace_nan(test)

In [7]:
train.head(2)

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,cat1,cat2,cat3
0,0,mlb cincinnati reds t shirt size xl,3,Men/Tops/T-shirts,missing,10.0,1,no description yet,Men,Tops,T-shirts
1,1,razer blackwidow chroma keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,this keyboard is in great condition and works ...,Electronics,Computers & Tablets,Components & Parts


In [8]:
#cleaning up string fields


#item_description
train['item_description'] = train.item_description.str.lower()
train['item_description'] = train.item_description.replace('[', ' ')
train['item_description'] = train.item_description.replace(']', ' ')
train['item_description'] = train.item_description.replace('[^0-9a-zA-Z]+', ' ', regex = True)
train['item_description'] = train.item_description.replace(" rm ", '', regex = True)
train['item_description'] = train.item_description.replace('[^a-zA-Z0-9 ]', '', regex = True)


#brand_names
train['brand_name'] = train.brand_name.str.lower()
train['brand_name'] = train.brand_name.replace('[^a-zA-Z0-9 ]', '', regex = True)


#item_names
train['name'] = train.name.str.lower()
train['name'] = train.name.replace('[', ' ')
train['name'] = train.name.replace(']', ' ')
train['name'] = train.name.replace('[^0-9a-zA-Z]+', ' ', regex = True)
train['name'] = train.name.replace(' rm ', '')
#train['name'] = train.name.replace('[^a-zA-Z0-9 ]', '', regex = True)


#item_condition_id
train['item_condition_id'] = train.item_condition_id.replace('5','4')


#categories
train['category_name'] = train.category_name.str.lower()
train['cat1'] = train.cat1.str.lower()
train['cat2'] = train.cat2.str.lower()
train['cat3'] = train.cat3.str.lower()
#train['cat4'] = train.cat4.str.lower()

In [9]:
#cleaning up string fields


#item_description
test['item_description'] = test.item_description.str.lower()
test['item_description'] = test.item_description.replace('[', ' ')
test['item_description'] = test.item_description.replace(']', ' ')
test['item_description'] = test.item_description.replace('[^0-9a-zA-Z]+', ' ', regex = True)
test['item_description'] = test.item_description.replace(" rm ", '', regex = True)
test['item_description'] = test.item_description.replace('[^a-zA-Z0-9 ]', '', regex = True)


#brand_names
test['brand_name'] = test.brand_name.str.lower()
test['brand_name'] = test.brand_name.replace('[^a-zA-Z0-9 ]', '', regex = True)


#item_names
test['name'] = test.name.str.lower()
test['name'] = test.name.replace('[', ' ')
test['name'] = test.name.replace(']', ' ')
test['name'] = test.name.replace('[^0-9a-zA-Z]+', ' ', regex = True)
test['name'] = test.name.replace(' rm ', '')
#train['name'] = train.name.replace('[^a-zA-Z0-9 ]', '', regex = True)


#item_condition_id
test['item_condition_id'] = test.item_condition_id.replace('5','4')


#categories
test['category_name'] = test.category_name.str.lower()
test['cat1'] = test.cat1.str.lower()
test['cat2'] = test.cat2.str.lower()
test['cat3'] = test.cat3.str.lower()
#train['cat4'] = train.cat4.str.lower()

In [10]:
train.head(2)

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,cat1,cat2,cat3
0,0,mlb cincinnati reds t shirt size xl,3,men/tops/t-shirts,missing,10.0,1,no description yet,men,tops,t-shirts
1,1,razer blackwidow chroma keyboard,3,electronics/computers & tablets/components & p...,razer,52.0,0,this keyboard is in great condition and works ...,electronics,computers & tablets,components & parts


# Extracting additional brands

In [11]:
#O(n) string search to extract brands
import ahocorasick

In [12]:
def string_search(dictionary, listStrings, listSubstrings):
    auto = ahocorasick.Automaton()
    for substr in listSubstrings:
        auto.add_word(substr, substr)
    auto.make_automaton()
    
    for idx, astr in enumerate(listStrings):
        seen = set()
        match_count = 0
        for end_ind, found in auto.iter(astr):
            if (found not in seen) and (match_count == 0):
                seen.add(found)
                #print(idx)
                #print(found+ "   " + astr)
                dictionary.update({idx: found})
                match_count +=1
#https://stackoverflow.com/questions/34816775/python-optimal-search-for-substring-in-list-of-strings
#http://ieva.rocks/2016/11/24/keyword-matching-with-aho-corasick/

In [13]:
#lookup table of missing brand names to index
missing_brand_lookup_train = pd.DataFrame(train[train['brand_name'] == 'missing'].reset_index().train_id)
missing_brand_lookup_train['join_key'] = missing_brand_lookup_train.index.values
#left join on train_id of output = index, and take the id number





#lookup table of missing brand names to index
missing_brand_lookup_test = pd.DataFrame(test[test['brand_name'] == 'missing'].reset_index().test_id)
missing_brand_lookup_test['join_key'] = missing_brand_lookup_test.index.values
#left join on train_id of output = index, and take the id number

In [14]:
#missing brands in the item name
name_strings_train =  train[train['brand_name'] ==
                        'missing'].reset_index().name.tolist() 
                        #string names where brand_name is null
name_strings_train = [item + ' ' for item in name_strings_train] # add ' ' to prevent missed brands at end of word



#missing brands in the description
desc_strings_train =  train[train['brand_name'] ==
                        'missing'].reset_index().item_description.tolist() 
                        #string names where brand_name is null
desc_strings_train = [item + ' ' for item in desc_strings_train] # add ' ' to prevent missed brands at end of word




#building list of existing brands
brand_substrings_train = train[train['brand_name'] != 
                        'missing'].reset_index().brand_name.tolist() 
                        #list of brand names where brand_name not null
    
brand_substrings_train = list(set(brand_substrings_train)) #de-duping list of brands

brand_substrings_train = [s for s in brand_substrings_train if len(s) > 2] #restricting brand substring length to > 2

if "all" in brand_substrings_train: brand_substrings_train.remove("all") #removing 'all' brand b/c too common of word
if "and" in brand_substrings_train: brand_substrings_train.remove("and") #removing 'all' brand b/c too common of word
if "pink" in brand_substrings_train: brand_substrings_train.remove("pink") #removing 'all' brand b/c too common of word
    
brand_substrings_train.sort(key=len, reverse=True) #sorting on longest string first

brand_substrings_train =  [item + ' ' for item in brand_substrings_train] #add ' ' to prevent misclassifications in middle of words





#initializing dictionaries to collect extracted brand names

brands_from_name_field_train = {}

brands_from_description_field_train = {}





In [15]:
#missing brands in the item name
name_strings_test =  test[test['brand_name'] ==
                        'missing'].reset_index().name.tolist() 
                        #string names where brand_name is null
name_strings_test = [item + ' ' for item in name_strings_test] # add ' ' to prevent missed brands at end of word



#missing brands in the description
desc_strings_test =  test[test['brand_name'] ==
                        'missing'].reset_index().item_description.tolist() 
                        #string names where brand_name is null
desc_strings_test = [item + ' ' for item in desc_strings_test] # add ' ' to prevent missed brands at end of word




#building list of existing brands
brand_substrings_test = test[test['brand_name'] != 
                        'missing'].reset_index().brand_name.tolist() 
                        #list of brand names where brand_name not null
    
brand_substrings_test = list(set(brand_substrings_test)) #de-duping list of brands

brand_substrings_test = [s for s in brand_substrings_test if len(s) > 2] #restricting brand substring length to > 2

if "all" in brand_substrings_test: brand_substrings_test.remove("all") #removing 'all' brand b/c too common of word
if "and" in brand_substrings_test: brand_substrings_test.remove("and") #removing 'all' brand b/c too common of word
if "pink" in brand_substrings_test: brand_substrings_test.remove("pink") #removing 'all' brand b/c too common of word
    
brand_substrings_test.sort(key=len, reverse=True) #sorting on longest string first

brand_substrings_test =  [item + ' ' for item in brand_substrings_test] #add ' ' to prevent misclassifications in middle of words





#initializing dictionaries to collect extracted brand names

brands_from_name_field_test = {}

brands_from_description_field_test = {}





In [16]:
#counting number of previous missing brands
previous_brand_count = train[train["brand_name"] != 'missing'].shape[0]
print("There are " + str(previous_brand_count) + " brands present in the training data")

There are 28416 brands present in the training data


In [17]:
print(previous_brand_count / train.shape[0])

0.56832


# brand extraction from item name

In [18]:
#brands hiding in name
string_search(brands_from_name_field_train, name_strings_train, brand_substrings_train)


#brands hiding in name
string_search(brands_from_name_field_test, name_strings_test, brand_substrings_test)

In [19]:
extracted_brands_train = pd.DataFrame(list(brands_from_name_field_train.items()))

extracted_brands_test = pd.DataFrame(list(brands_from_name_field_test.items()))

In [20]:
extracted_brands_train.columns = ['train_id', 'brand_name']

extracted_brands_train['brand_name'] = extracted_brands_train['brand_name'].map(lambda x: str(x)[:-1])

extracted_brands_test.columns = ['test_id', 'brand_name']

extracted_brands_test['brand_name'] = extracted_brands_test['brand_name'].map(lambda x: str(x)[:-1])

In [21]:
extracted_brands_train = pd.merge(extracted_brands_train,missing_brand_lookup_train, left_on='train_id', right_on='join_key', how='left')


extracted_brands_test = pd.merge(extracted_brands_test,missing_brand_lookup_test, left_on='test_id', right_on='join_key', how='left')

In [22]:
train = pd.merge(train, extracted_brands_train, how='left', left_on='train_id', right_on='train_id_y')
train.brand_name_y = train.brand_name_y.fillna('missing')

test = pd.merge(test, extracted_brands_test, how='left', left_on='test_id', right_on='test_id_y')
test.brand_name_y = test.brand_name_y.fillna('missing')

In [23]:
train['brand_name'] = train[['brand_name_x','brand_name_y']].apply(
    lambda x: x['brand_name_y'] 
    if x['brand_name_x'] == 'missing' 
    else x['brand_name_x'], axis=1)


test['brand_name'] = test[['brand_name_x','brand_name_y']].apply(
    lambda x: x['brand_name_y'] 
    if x['brand_name_x'] == 'missing' 
    else x['brand_name_x'], axis=1)

In [24]:
train.drop('brand_name_x', axis=1, inplace=True)
train.drop('brand_name_y', axis=1, inplace=True)
train.drop('train_id_x', axis=1, inplace=True)
train.drop('train_id_y', axis=1, inplace=True)
train.drop('join_key', axis=1, inplace=True)


test.drop('brand_name_x', axis=1, inplace=True)
test.drop('brand_name_y', axis=1, inplace=True)
test.drop('test_id_x', axis=1, inplace=True)
test.drop('test_id_y', axis=1, inplace=True)
test.drop('join_key', axis=1, inplace=True)

In [25]:
train.tail(2)

,train_id,name,item_condition_id,category_name,price,shipping,item_description,cat1,cat2,cat3,brand_name
49998,49998,rm 2 bundles for this price,1,NaN,75.0,1,2 bundle any length free shipping same day sh...,,,,virgin only
49999,49999,vs pink bling v neck tshirt sz s,1,women/tops & blouses/t-shirts,50.0,1,price firm don t ask for the lowest price pls ...,women,tops & blouses,t-shirts,victorias secret


In [26]:
print("Extracted " + str(train[train["brand_name"] != 'missing'].shape[0]  - previous_brand_count) + " brands from item name in training data")
previous_brand_count = train[train["brand_name"] != 'missing'].shape[0]

Extracted 5751 brands from item name in training data


# brand extraction from item description

In [27]:
#brands hiding in description
string_search(brands_from_description_field_train, desc_strings_train, brand_substrings_train)


#brands hiding in description
string_search(brands_from_description_field_test, desc_strings_test, brand_substrings_test)

In [28]:
extracted_brands_train = pd.DataFrame(list(brands_from_description_field_train.items()))
extracted_brands_train.columns = ['train_id', 'brand_name']

extracted_brands_train['brand_name'] = extracted_brands_train['brand_name'].map(lambda x: str(x)[:-1])
extracted_brands_train = pd.merge(extracted_brands_train,missing_brand_lookup_train, left_on='train_id', right_on='join_key', how='left')


extracted_brands_test = pd.DataFrame(list(brands_from_description_field_test.items()))
extracted_brands_test.columns = ['test_id', 'brand_name']

extracted_brands_test['brand_name'] = extracted_brands_test['brand_name'].map(lambda x: str(x)[:-1])
extracted_brands_test = pd.merge(extracted_brands_test,missing_brand_lookup_test, left_on='test_id', right_on='join_key', how='left')

In [29]:
train = pd.merge(train, extracted_brands_train, how='left', left_on='train_id', right_on='train_id_y')
train.brand_name_y = train.brand_name_y.fillna('missing')

train['brand_name'] = train[['brand_name_x','brand_name_y']].apply(
    lambda x: x['brand_name_y'] 
    if x['brand_name_x'] == 'missing' 
    else x['brand_name_x'], axis=1)
#testing
#train[(train['brand_name_x'] == 'missing') & (train['brand_name'] != 'missing')]




test = pd.merge(test, extracted_brands_test, how='left', left_on='test_id', right_on='test_id_y')
test.brand_name_y = test.brand_name_y.fillna('missing')

test['brand_name'] = test[['brand_name_x','brand_name_y']].apply(
    lambda x: x['brand_name_y'] 
    if x['brand_name_x'] == 'missing' 
    else x['brand_name_x'], axis=1)
#testing
#train[(train['brand_name_x'] == 'missing') & (train['brand_name'] != 'missing')]

In [30]:
train.drop('brand_name_x', axis=1, inplace=True)
train.drop('brand_name_y', axis=1, inplace=True)
train.drop('train_id_x', axis=1, inplace=True)
train.drop('train_id_y', axis=1, inplace=True)
train.drop('join_key', axis=1, inplace=True)

test.drop('brand_name_x', axis=1, inplace=True)
test.drop('brand_name_y', axis=1, inplace=True)
test.drop('test_id_x', axis=1, inplace=True)
test.drop('test_id_y', axis=1, inplace=True)
test.drop('join_key', axis=1, inplace=True)

In [31]:
test.tail(2)

,test_id,name,item_condition_id,category_name,shipping,item_description,cat1,cat2,cat3,brand_name
49998,49998,free ship new vs bombshell 2 5oz mist,1,beauty/fragrance/women,1,free ship new vs bombshell 2 5oz mist,beauty,fragrance,women,victorias secret
49999,49999,fox beanie,3,women/women's accessories/hats,0,flips inside out,women,women's accessories,hats,fox racing


In [32]:
print("Extracted " + str(train[train["brand_name"] != 'missing'].shape[0]  - previous_brand_count) + " brands from item desc in training data")

Extracted 4222 brands from item desc in training data


In [33]:
print(train[train.brand_name != 'missing'].shape[0] / train.shape[0])

0.76778


# Grouping data by category and brand

In [34]:
train['brand_name'] = train.brand_name.replace('missing', np.nan)

brand_summary = train.groupby(['cat1','cat2','cat3','brand_name']).agg({'count'})['price']

In [35]:
#brand_summary.head(50)

In [36]:
#renaming columns
brand_summary.rename(columns={'count':'brand_item_count'}, inplace=True)

brand_summary = brand_summary[brand_summary.brand_item_count>100].\
                sort_values(by='brand_item_count', ascending=False).reset_index().copy()

In [37]:
brand_summary.tail(5)

,cat1,cat2,cat3,brand_name,brand_item_count
31,women,athletic apparel,"pants, tights, leggings",nike,106
32,women,shoes,boots,ugg australia,105
33,women,shoes,fashion sneakers,converse,105
34,women,athletic apparel,"pants, tights, leggings",independent,105
35,women,athletic apparel,shirts & tops,lululemon,105


In [38]:
#generating random model names for each category
brand_model_num = pd.Series(['brand_'+str(np.random.randint(1,len(brand_summary)*10000)) for i in 
                            range(len(brand_summary))],name='brand_model_num',dtype=str)

brand_summary = pd.concat([brand_summary,brand_model_num], axis=1)

brand_summary.head()

,cat1,cat2,cat3,brand_name,brand_item_count,brand_model_num
0,women,athletic apparel,"pants, tights, leggings",lularoe,895,brand_75030
1,electronics,video games & consoles,games,nintendo,337,brand_277315
2,women,underwear,bras,victorias secret,331,brand_205386
3,women,tops & blouses,t-shirts,pink,304,brand_194290
4,women,shoes,athletic,nike,289,brand_160206


In [39]:
train= pd.merge(train,brand_summary,how='left',on=['cat1','cat2','cat3','brand_name'])



test= pd.merge(test,brand_summary,how='left',on=['cat1','cat2','cat3','brand_name'])

In [40]:
#train.head(20)

In [41]:
#looking at train dataset with categories

In [42]:
#simple look at train dataset with categories
train.loc[train.brand_model_num != None, ['cat1','cat2','cat3','brand_name','brand_model_num','price']].\
groupby(['cat1','cat2','cat3','brand_name','brand_model_num']).agg({pd.Series.count})['price'].\
reset_index().sort_values(by='count', ascending=False).head(5)

,cat1,cat2,cat3,brand_name,brand_model_num,count
14,women,athletic apparel,"pants, tights, leggings",lularoe,brand_75030,895.0
4,electronics,video games & consoles,games,nintendo,brand_277315,337.0
34,women,underwear,bras,victorias secret,brand_205386,331.0
31,women,tops & blouses,t-shirts,pink,brand_194290,304.0
25,women,shoes,athletic,nike,brand_160206,289.0


In [43]:
#uncategorized items, grouping at category level, where count of items >  100

In [44]:
cat3_summary = train[pd.isnull(train.brand_model_num) == True].groupby(['cat1','cat2','cat3'])\
                .agg({'count'})['price']
    
cat3_summary.rename(columns={'count':'cat3_item_count'},inplace = True)

cat3_summary = cat3_summary[cat3_summary.cat3_item_count>100].\
                sort_values(by='cat3_item_count',ascending=False).reset_index().copy()

In [45]:
#generating random model names for each category

cat3_model_num = pd.Series(['cat3_'+str(np.random.randint(1,len(cat3_summary)*10000)) for i in
                           range(len(cat3_summary))], name='cat3_model_num',dtype = str)

cat3_summary = pd.concat([cat3_summary.reset_index(),cat3_model_num], axis = 1)

In [46]:
cat3_summary.head()

,index,cat1,cat2,cat3,cat3_item_count,cat3_model_num
0,0,beauty,makeup,face,1161,cat3_683208
1,1,women,tops & blouses,t-shirts,1049,cat3_9679
2,2,beauty,makeup,lips,923,cat3_1009294
3,3,beauty,makeup,eyes,881,cat3_954812
4,4,women,tops & blouses,"tank, cami",713,cat3_420333


In [47]:
train = pd.merge(train,cat3_summary,how='left',on=['cat1','cat2','cat3'])

test = pd.merge(test,cat3_summary, how = 'left', on = ['cat1','cat2','cat3'])

In [48]:
train['final_model_num'] = train.apply(lambda x: x['cat3_model_num'] if pd.isnull(x['brand_model_num']) == True
                                      else x['brand_model_num'], axis = 1)


test['final_model_num'] = test.apply(lambda x: x['cat3_model_num'] if pd.isnull(x['brand_model_num']) == True
                                      else x['brand_model_num'], axis = 1)

In [49]:
train['final_item_count'] = train.apply(lambda x: x['cat3_item_count'] if pd.isnull(x['brand_item_count']) == True
                                       else x['brand_item_count'], axis = 1)


train.drop(['brand_item_count','brand_model_num','index', 'cat3_item_count','cat3_model_num'], axis = 1, inplace = True)



test['final_item_count'] = test.apply(lambda x: x['cat3_item_count'] if pd.isnull(x['brand_item_count']) == True
                                       else x['brand_item_count'], axis = 1)


test.drop(['brand_item_count','brand_model_num','index', 'cat3_item_count','cat3_model_num'], axis = 1, inplace = True)

In [50]:
train.head(3)

,train_id,name,item_condition_id,category_name,price,shipping,item_description,cat1,cat2,cat3,brand_name,final_model_num,final_item_count
0,0,mlb cincinnati reds t shirt size xl,3,men/tops/t-shirts,10.0,1,no description yet,men,tops,t-shirts,mlb,cat3_280819,519.0
1,1,razer blackwidow chroma keyboard,3,electronics/computers & tablets/components & p...,52.0,0,this keyboard is in great condition and works ...,electronics,computers & tablets,components & parts,razer,NaN,NaN
2,2,ava viv blouse,1,women/tops & blouses/blouse,10.0,1,adorable top with a hint of lace and a key hol...,women,tops & blouses,blouse,target,cat3_691436,682.0


In [51]:
print(train.isnull().sum())

train_id                 0
name                     0
item_condition_id        0
category_name          237
price                    0
shipping                 0
item_description         0
cat1                     0
cat2                     0
cat3                     0
brand_name           11611
final_model_num      13013
final_item_count     13013
dtype: int64


In [52]:
#train.brand_name.fillna(value = "missing", inplace = True)
#train.category_name.fillna(value="missing", inplace = True)
#train[train.brand_name == 'missing']

In [53]:
train[train.brand_name == 'victorias secret']

,train_id,name,item_condition_id,category_name,price,shipping,item_description,cat1,cat2,cat3,brand_name,final_model_num,final_item_count
11,11,new vs pi k body mists,1,beauty/fragrance/women,34.0,0,5 new vs pink body mists 2 5 oz each fresh cl...,beauty,fragrance,women,victorias secret,brand_68414,206.0
21,21,nwt vs ultimate sports bra 34ddd,1,women/athletic apparel/sports bras,20.0,0,nwt victoria s secret ultimate sport bra maxim...,women,athletic apparel,sports bras,victorias secret,cat3_204890,291.0
39,39,victoria secret 34 c corest top,3,women/underwear/bras,10.0,0,victoria secret 34 c corest top will bundle to...,women,underwear,bras,victorias secret,brand_205386,331.0
111,111,nwt victoria secret bathingsuit bottoms,1,women/swimwear/two-piece,26.0,0,size small nwt bathing suit bottoms,women,swimwear,two-piece,victorias secret,brand_134389,116.0
118,118,pink body mist,3,beauty/fragrance/women,16.0,1,bombshell eau so sexy bombshell was used a bit...,beauty,fragrance,women,victorias secret,brand_68414,206.0
178,178,vs pink 1 2zip collared sz m,1,women/sweaters/collared,47.0,1,price firm don t ask me for the lowest price p...,women,sweaters,collared,victorias secret,NaN,NaN
182,182,victoria s secret pink night shirt,2,women/tops & blouses/other,12.0,0,from victoria s secret pink plaid and white pa...,women,tops & blouses,other,victorias secret,NaN,NaN
190,190,victoria s secret small bikini top,2,women/swimwear/two-piece,9.0,0,in brand new condition just missing the strap,women,swimwear,two-piece,victorias secret,brand_134389,116.0
226,226,victoria s secret pj set size med,3,women/other/other,18.0,0,victoria s secret hot pink pj set with tank to...,women,other,other,victorias secret,cat3_17373,263.0
386,386,victoria s secret swim suit,3,women/swimwear/two-piece,25.0,0,good used condition,women,swimwear,two-piece,victorias secret,brand_134389,116.0


In [54]:
train.brand_name.fillna(value = "missing", inplace = True)
train.category_name.fillna(value="missing", inplace = True)

test.brand_name.fillna(value = "missing", inplace = True)
test.category_name.fillna(value="missing", inplace = True)

# Removing brand noise from item descriptions and names where model is at brand level

# Modeling

In [55]:
#3 part plan to improve model: 
#1 - missing models < 100 and models w no categories. model is predicting no categories, not predicting < 100
#2 - running entire model with testing data and submitting kernel.
#3 - improving poor categories with mean/median?

In [56]:
#stop word list to remove
my_stopword_list = ['and','to','the','of', 'a','are','for','you','or','it']

In [57]:
#testing mse
import math
def rmsle(y_test, y_pred):
    assert len(y_test) == len(y_pred)
    to_sum = [(math.log(y_pred[i] + 1) - math.log(y_test[i] + 1)) ** 2.0 for i,pred in enumerate(y_pred)]
    return (sum(to_sum) * (1.0/len(y_test))) ** 0.5

In [58]:
classifiers = [
    #svm.SVR(),
    Ridge(alpha=2, random_state=241),
    linear_model.SGDRegressor(),
    #linear_model.BayesianRidge(),
    #linear_model.LassoLars(),
    #linear_model.ARDRegression(),
    linear_model.PassiveAggressiveRegressor(),
    #linear_model.TheilSenRegressor(),
    linear_model.LinearRegression()
    ]

vectorizer = TfidfVectorizer(ngram_range = (1,2), stop_words=my_stopword_list)#, max_features = 50)
#http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [59]:
#extracting model number where categories are null, so that we can use brands as features
model_no_cat_train = str(set(train[train.category_name == 'missing']['final_model_num'].values))

#extracting model number where categories are null, so that we can use brands as features
model_no_cat_test = str(set(test[test.category_name == 'missing']['final_model_num'].values))

In [60]:
train.head(2)

,train_id,name,item_condition_id,category_name,price,shipping,item_description,cat1,cat2,cat3,brand_name,final_model_num,final_item_count
0,0,mlb cincinnati reds t shirt size xl,3,men/tops/t-shirts,10.0,1,no description yet,men,tops,t-shirts,mlb,cat3_280819,519.0
1,1,razer blackwidow chroma keyboard,3,electronics/computers & tablets/components & p...,52.0,0,this keyboard is in great condition and works ...,electronics,computers & tablets,components & parts,razer,NaN,NaN


In [61]:
train['target'] = np.log1p(train['price'])

In [62]:
#print('5 folds scaling the test_df')
#print(test.shape)
#test_len = test.shape[0]
#def simulate_test(test):
#    if test.shape[0] < 800000:
#        indices = np.random.choice(test.index.values, 2800000)
#        test_ = pd.concat([test, test.iloc[indices]], axis=0)
#        return test_.copy()
#    else:
#        return test
#test = simulate_test(test)
#print('new shape ', test.shape)

In [63]:
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [64]:
from keras.preprocessing.sequence import pad_sequences

def get_keras_data(dataset):
    X = {
        'name': pad_sequences(dataset.seq_name, maxlen=MAX_NAME_SEQ)
        ,'item_desc': pad_sequences(dataset.seq_item_description
                                    , maxlen=MAX_ITEM_DESC_SEQ)
        #,'brand': np.array(dataset.brand)
        #,'category': np.array(dataset.category)
        #,'category_name': pad_sequences(dataset.seq_category_name
        #                                , maxlen=MAX_CATEGORY_NAME_SEQ)
        ,'item_condition': np.array(dataset.item_condition_id)
        ,'num_vars': np.array(dataset[["shipping"]])
    }
    return X

#KERAS MODEL DEFINITION
from keras.layers import Input, Dropout, Dense, BatchNormalization, \
    Activation, concatenate, GRU, Embedding, Flatten
from keras.models import Model
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping#, TensorBoard
from keras import backend as K
from keras import optimizers
from keras import initializers

In [74]:
def get_model():
    #params
    dr_r = dr
    
    #Inputs
    name = Input(shape=[X_train["name"].shape[1]], name="name")
    item_desc = Input(shape=[X_train["item_desc"].shape[1]], name="item_desc")
    #brand = Input(shape=[1], name="brand")
    #category = Input(shape=[1], name="category")
    #category_name = Input(shape=[X_train["category_name"].shape[1]], 
    #                      name="category_name")
    item_condition = Input(shape=[1], name="item_condition")
    num_vars = Input(shape=[X_train["num_vars"].shape[1]], name="num_vars")
    
    #Embeddings layers
    emb_size = 60
    
    emb_name = Embedding(MAX_TEXT, emb_size//3)(name)
    emb_item_desc = Embedding(MAX_TEXT, emb_size)(item_desc)
    #emb_category_name = Embedding(MAX_TEXT, emb_size//3)(category_name)
    #emb_brand = Embedding(MAX_BRAND, 10)(brand)
    #emb_category = Embedding(MAX_CATEGORY, 10)(category)
    emb_item_condition = Embedding(MAX_CONDITION, 5)(item_condition)
    
    rnn_layer1 = GRU(16) (emb_item_desc)
    #rnn_layer2 = GRU(8) (emb_category_name)
    rnn_layer2 = GRU(8) (emb_name)
    
    #main layer
    main_l = concatenate([
        #Flatten() (emb_brand)
        #, Flatten() (emb_category)
          Flatten() (emb_item_condition)
        , rnn_layer1
        , rnn_layer2
        #, rnn_layer3
        , num_vars
    ])
    main_l = Dropout(0.3)(Dense(512,activation='relu') (main_l))
    main_l = Dropout(0.2)(Dense(88,activation='relu') (main_l))
    
    #output
    output = Dense(1,activation="linear") (main_l)
    
    #model
    model = Model([name, item_desc
                   , item_condition, num_vars], output)
    #optimizer = optimizers.RMSprop()
    optimizer = optimizers.Adam()
    model.compile(loss="mse", 
                  optimizer=optimizer)
    return model


In [ ]:
def eval_model(model):
    val_preds = model.predict(X_valid)
    val_preds = np.expm1(val_preds)
    
    y_true = np.array(dvalid.price.values)
    y_pred = val_preds[:, 0]
    v_rmsle = rmsle(y_true, y_pred)
    print(" RMSLE error on dev test: "+str(v_rmsle))
    return v_rmsle

In [75]:
import gc

In [76]:
dr = 0.25

In [77]:
exp_decay = lambda init, fin, steps: (init/fin)**(1/(steps-1)) - 1

In [78]:
from sklearn.model_selection import train_test_split, cross_val_score

In [85]:
test_id = []
price_prediction = []

np_array_train = train.final_model_num.unique()
np_clean_train = [x for x in np_array_train if str(x) != 'nan']

np_array_test = test.final_model_num.unique()
np_clean_test = [x for x in np_array_test if str(x) != 'nan']

for model in np_clean_train:
    
    
    if model == None:
        continue
    else:
        print(model)
        train1 = train[train.final_model_num == model]
        
        if test[test.final_model_num == model].shape[0] == 0:
            continue
        else:
            test1 = test[test.final_model_num == model]
        
        
            raw_text = np.hstack([
                                  train1.item_description.str.lower(), 
                                  train1.name.str.lower()])

            tok_raw = Tokenizer()
            tok_raw.fit_on_texts(raw_text)
            train1["seq_item_description"] = tok_raw.texts_to_sequences(train1.item_description.str.lower())
            test1["seq_item_description"] = tok_raw.texts_to_sequences(test1.item_description.str.lower())
            train1["seq_name"] = tok_raw.texts_to_sequences(train1.name.str.lower())
            test1["seq_name"] = tok_raw.texts_to_sequences(test1.name.str.lower())

            dtrain, dvalid = train_test_split(train1, random_state=666, train_size=0.7)

            #EMBEDDINGS MAX VALUE
            #Base on the histograms, we select the next lengths
            MAX_NAME_SEQ = 20 #17
            MAX_ITEM_DESC_SEQ = 60 #269
            #MAX_CATEGORY_NAME_SEQ = 20 #8
            MAX_TEXT = np.max([np.max(train1.seq_name.max())
                               , np.max(test1.seq_name.max())
                               , np.max(train1.seq_item_description.max())
                               , np.max(test1.seq_item_description.max())])+2
            #MAX_CATEGORY = np.max([train.category.max(), test.category.max()])+1
            #MAX_BRAND = np.max([train.brand.max(), test.brand.max()])+1
            MAX_CONDITION = np.max([train1.item_condition_id.max(), 
                                    test1.item_condition_id.max()])+1

            X_train = get_keras_data(dtrain)
            X_valid = get_keras_data(dvalid)
            X_test = get_keras_data(test1)

            epochs = 2
            BATCH_SIZE = train1.shape[0]
            steps = int(len(X_train['name'])/BATCH_SIZE) * epochs
            lr_init, lr_fin = 0.009, 0.006
            lr_decay = exp_decay(lr_init, lr_fin, steps)
            log_subdir = '_'.join(['ep', str(epochs),
                                'bs', str(BATCH_SIZE),
                                'lrI', str(lr_init),
                                'lrF', str(lr_fin),
                                'dr', str(dr)])

            nn = get_model()
            K.set_value(nn.optimizer.lr, lr_init)
            K.set_value(nn.optimizer.decay, lr_decay)

            history = nn.fit(X_train, dtrain.target
                                , epochs=epochs
                                , batch_size=BATCH_SIZE
                                , validation_split=0.3
                                , verbose=10
                                )

            #EVLUEATE THE MODEL ON DEV TEST
            v_rmsle = eval_model(nn)

            preds = nn.predict(X_test, batch_size=BATCH_SIZE)
            preds = np.expm1(preds)
            print('Finished predicting test set...')
            test1['predicted_price'] = preds
            
            #submission = test1[["test_id"]][:test_len]
            #submission["price"] = preds[:test_len]*0.9
            #print('[{}] Finished predicting test set...'.format(time.time() - start_time))
            
            
            test_id.extend(test1.test_id.values.tolist())
            price_prediction.extend(test1.predicted_price.tolist())
            gc.collect()           
    

cat3_280819


/Users/Jpeteyy/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Jpeteyy/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Jpeteyy/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/

Train on 254 samples, validate on 109 samples
Epoch 1/2


InvalidArgumentError: indices[128,17] = 2049 is not in [0, 2048)
	 [[Node: embedding_19/Gather = Gather[Tindices=DT_INT32, Tparams=DT_FLOAT, validate_indices=true, _device="/job:localhost/replica:0/task:0/device:CPU:0"](embedding_19/embeddings/read, embedding_19/Cast)]]

Caused by op 'embedding_19/Gather', defined at:
  File "/Users/Jpeteyy/anaconda/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/Jpeteyy/anaconda/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/Jpeteyy/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/Jpeteyy/anaconda/lib/python3.5/site-packages/traitlets/config/application.py", line 653, in launch_instance
    app.start()
  File "/Users/Jpeteyy/anaconda/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/Users/Jpeteyy/anaconda/lib/python3

In [ ]:
train.final_model_num.fillna(value="missing", inplace = True)

In [ ]:
no_model_no_brand = train[(train.final_model_num == 'missing') & (train.brand_name == 'missing')]\
                    .groupby(['cat1','cat2','cat3'])['price'].mean().reset_index().copy()
    
no_model_no_brand.rename(columns={'price': 'median_price'}, inplace=True)

#no_model_no_brand['final_model_num_no_mod_no_brnd'] = pd.Series(['cat3_'+str(np.random.randint(1,len(no_model_no_brand)*10000)) for i in
#                           range(len(no_model_no_brand))], dtype = str)

In [ ]:
no_model_w_brand = train[(train.final_model_num == 'missing') & (train.brand_name != 'missing')]\
                    .groupby(['cat1','cat2','cat3','brand_name'])['price'].mean().reset_index().copy()

no_model_w_brand.rename(columns={'price':'median_price'},inplace = True)

#no_model_w_brand ['final_model_num_no_mod_w_brnd'] = pd.Series(['cat3_'+str(np.random.randint(1,len(no_model_w_brand)*10000)) for i in
#                           range(len(no_model_w_brand))], dtype = str)

In [ ]:
train = pd.merge(train, no_model_no_brand, how= 'left',on = ['cat1','cat2','cat3'])
train = pd.merge(train, no_model_w_brand, how= 'left',on = ['cat1','cat2','cat3','brand_name'])

test = pd.merge(test, no_model_no_brand, how = 'left', on = ['cat1','cat2','cat3'])
test = pd.merge(test, no_model_w_brand, how = 'left', on = ['cat1','cat2','cat3','brand_name'])

In [ ]:
#train.drop(['median_price'], axis = 1, inplace = True)

In [ ]:
train['median_price'] = train.apply(lambda x: x['median_price_x'] if (x['final_model_num'] == 'missing'
                                        and str(x['median_price_y']) == 'nan')
                                      else x['median_price_y']
                                        , axis = 1)

#train['median_price'] = train.median_price_x.combine_first(train.median_price_y)

train.drop(['median_price_x','median_price_y'], inplace = True, axis = 1)          


test['median_price'] = test.apply(lambda x: x['median_price_x'] if (x['final_model_num'] == 'missing'
                                        and str(x['median_price_y']) == 'nan')
                                      else x['median_price_y']
                                        , axis = 1)

#train['median_price'] = train.median_price_x.combine_first(train.median_price_y)   

test.drop(['median_price_x','median_price_y'], inplace = True, axis = 1)                                      

In [ ]:
#to do's:
#median/avg for non brand or category items
#submit kernel
#nn, sentiment/meaning, word2vec
#median or avg to replace worst 
#one hot encoding dummies for brand name in categories, test submission or before/after error in a new column? % diff?
#???

In [ ]:
test_ids = np.array(test_id)
price_predictions = np.array(price_prediction)

predicted_cats_brands = pd.DataFrame({'test_id' : test_id, 'price_prediction' : price_predictions})

In [ ]:
predicted_cats_brands.head(5)

In [ ]:
test.head(5)

In [ ]:
test_final = pd.merge(test[['test_id','median_price']],predicted_cats_brands, how = 'left', on = 'test_id' )


In [ ]:
test_final['price_prediction'] = test_final.price_prediction.combine_first(test_final.median_price)  

In [ ]:
test_final.price_prediction.fillna(value=10, inplace=True)

In [ ]:
test_final.rename(columns={'price_prediction':'price'}, inplace=True)

In [ ]:
test_final.head()

In [ ]:
test_final[['test_id','price']].to_csv('/Users/Jpeteyy/Desktop/mercari_sub.csv', index = False)

In [ ]:
test_final.head()